In [1]:
import sqlite3
import os

# Xóa Database cũ

In [2]:
db_path = 'warehouse.db'

if os.path.exists(db_path):
    os.remove(db_path)
    print(f"Đã xóa database: {db_path}")
else:
    print(f"Không tìm thấy file database: {db_path}")

Đã xóa database: warehouse.db


# Tạo database

In [3]:
conn = sqlite3.connect('warehouse.db')
cursor = conn.cursor()

# Bật khóa ngoại
cursor.execute("PRAGMA foreign_keys = ON;")

conn.commit()
conn.close()

In [4]:
create_database = """
-- SQLite is file-based, so CREATE DATABASE and USE statements are not needed.
-- The database is created when you first connect to a database file.

-- 1. Create group_mc table
-- Referenced by 'machine' and 'machine_pos'
CREATE TABLE group_mc (
    id INTEGER PRIMARY KEY, -- SQLite often uses INTEGER PRIMARY KEY for auto-incrementing IDs
    mc_name VARCHAR(255)
);

-- 2. Create machine_type table
-- Referenced by 'spare_parts'
CREATE TABLE machine_type (
    id INTEGER PRIMARY KEY, -- SQLite often uses INTEGER PRIMARY KEY for auto-incrementing IDs
    machine VARCHAR(255)
);

-- 3. Create machine table
-- References 'group_mc'
CREATE TABLE machine (
    name VARCHAR(255) PRIMARY KEY,
    group_mc_id INTEGER,
    FOREIGN KEY (group_mc_id) REFERENCES group_mc(id)
);

-- 4. Create employees table
-- Referenced by 'import_export'
-- Including columns identified in the original UPDATE statements across both turns
CREATE TABLE employees (
    amann_id VARCHAR(255) PRIMARY KEY,
    name VARCHAR(255),
    title VARCHAR(255),
    level VARCHAR(255),
    active INTEGER, -- BOOLEAN maps to INTEGER (0 for FALSE, 1 for TRUE)
    birthday TEXT, -- date maps to TEXT (ISO8601 format 'YYYY-MM-DD')
    start_date TEXT, -- date maps to TEXT (ISO8601 format 'YYYY-MM-DD')
    check_in_time TEXT, -- time maps to TEXT (ISO8601 format 'HH:MM:SS')
    check_out_time TEXT, -- time maps to TEXT (ISO8601 format 'HH:MM:SS')
    address VARCHAR(255),
    phone_number VARCHAR(50),
    email VARCHAR(255),
    gender VARCHAR(10),
    years_worked INTEGER, -- int maps to INTEGER
    age INTEGER -- Adding age column based on UPDATEs
);

-- 5. Create machine_pos table
-- References 'group_mc', Referenced by 'import_export'
-- Correcting the primary key definition based on the ALTER TABLEs and INSERT data structure
CREATE TABLE machine_pos (
    mc_pos VARCHAR(255) PRIMARY KEY, -- Setting mc_pos as the primary key based on ALTER TABLEs
    mc_id INTEGER, -- Assuming mc_id links to group_mc as seen in INSERTs
    FOREIGN KEY (mc_id) REFERENCES group_mc(id) -- Added FK relationship based on INSERT data
);

-- 6. Create spare_parts table
-- References 'machine_type', Referenced by 'import_export'
-- Including all columns from the original CREATE TABLE and the subsequent ALTER TABLEs/INSERTs
CREATE TABLE spare_parts (
    material_no VARCHAR(255) PRIMARY KEY,
    part_no VARCHAR(255),
    description TEXT,
    machine_type_id INTEGER, -- INT maps to INTEGER
    bin VARCHAR(50),
    cost_center VARCHAR(255),
    price REAL, -- DECIMAL(10, 2) maps to REAL or NUMERIC
    stock INTEGER, -- INT maps to INTEGER
    safety_stock INTEGER, -- INT maps to INTEGER
    safety_stock_check INTEGER, -- BOOLEAN maps to INTEGER (0 or 1)
    image_url VARCHAR(255), -- Added from ALTER TABLE in first turn
    import_date TEXT, -- datetime maps to TEXT (ISO8601 format)
    export_date TEXT, -- datetime maps to TEXT (ISO8601 format)
    FOREIGN KEY (machine_type_id) REFERENCES machine_type(id)
);

-- 7. Create import_export table
-- References 'spare_parts', 'machine_pos', and 'employees'
CREATE TABLE import_export (
    part_id VARCHAR(255),
    quantity INTEGER, -- INT maps to INTEGER
    mc_pos_id VARCHAR(255),
    empl_id VARCHAR(255),
    date TEXT, -- DATE maps to TEXT (ISO8601 format 'YYYY-MM-DD')
    reason TEXT,
    im_ex_flag INTEGER, -- BOOLEAN maps to INTEGER (0 or 1)
    FOREIGN KEY (part_id) REFERENCES spare_parts(material_no),
    FOREIGN KEY (mc_pos_id) REFERENCES machine_pos(mc_pos),
    FOREIGN KEY (empl_id) REFERENCES employees(amann_id)
);

-- 8. Create index on machine_pos(mc_pos)
-- This should be done after the table is created
CREATE INDEX idx_mc_pos ON machine_pos(mc_pos);

-- The original SELECT * FROM machine_type; is a query, not a definition, and is omitted.
-- The original ALTER TABLE machine_pos DROP PRIMARY KEY; and ALTER TABLE machine_pos ADD PRIMARY KEY (mc_pos);
-- were handled by defining the machine_pos table correctly initially with mc_pos as PK.
"""

# Insert dữ liệu

In [5]:
conn = sqlite3.connect('warehouse.db')
cursor = conn.cursor()


# Chạy toàn bộ script một lần
cursor.executescript(create_database)

conn.commit()
conn.close()

In [6]:
insert_database = """
-- Insert data into group_mc table
INSERT INTO group_mc (id, mc_name)
VALUES
(1, 'Chang Yi'),
(2, 'Fong'),
(3, 'SSM'),
(4, 'Graf'),
(5, 'Ugolini'),
(6, 'Tecnorama'),
(7, 'Fadis'),
(8, 'DETTIN');

-- Insert data into group_mc table
INSERT INTO machine_type (id, machine)
VALUES
(1, 'Chang Yi'),
(2, 'Fong'),
(3, 'SSM'),
(4, 'Graf'),
(5, 'Ugolini'),
(6, 'Tecnorama'),
(7, 'Fadis'),
(8, 'DETTIN');

-- Insert data into machine table
INSERT INTO machine (name, group_mc_id)
VALUES
('Chang Yi-1', 1),
('Chang Yi-2', 1),
('Chang Yi-3', 1),
('Chang Yi-4', 1),
('Chang Yi-5', 1),
('Fong-1', 2),
('Fong-2', 2),
('Fong-3', 2),
('Fong-4', 2),
('SSM-1', 3),
('SSM-2', 3),
('SSM-3', 3),
('SSM-4', 3),
('Graf-1', 4),
('Graf-2', 4),
('Graf-3', 4),
('Graf-4', 4),
('Ugolini-1', 5),
('Ugolini-2', 5),
('Ugolini-3', 5),
('Tecnorama-1', 6),
('Tecnorama-2', 6),
('Fadis-1', 7),
('Fadis-2', 7),
('DETTIN-1', 8);

-- Insert data into spare_parts table
INSERT INTO spare_parts (
    material_no, part_no, description, machine_type_id, bin, cost_center, price, stock, safety_stock, safety_stock_check
) VALUES
('SP001', 'P001', 'M0F67-MECHANICALSEAL', 1, 'A1', 'D.10', 50.00, 10, 20, 1), -- TRUE becomes 1 in SQLite
('SP002', 'P002', 'MF86E-SL2HOSESCREWONCONNECTIONWITHMALETHREAD618', 2, 'A2', 'CC002', 75.00, 150, 30, 0), -- FALSE becomes 0
('SP003', 'P003', 'MC1E9-SL2CERAMICSPLITEYELETD68H6D28MM', 1, 'B1', 'D.11', 20.00, 60, 15, 1), -- TRUE becomes 1
('SP004', 'P004', 'MC29E-SL2AGCERAMICSPLITEYELETD65H68D38MM', 1, 'B2', 'CC003', 25.00, 80, 20, 1), -- TRUE becomes 1
('SP005', 'P005', 'MD7A5-SL2PUGLASSCOVERGEARS', 2, 'B3', 'CC004', 18.00, 40, 10, 0), -- FALSE becomes 0
('SP006', 'P006', 'M1E71-SL2SEALRUBBER', 2, 'B4', 'D.12', 12.50, 90, 25, 0), -- FALSE becomes 0
('SP007', 'P007', 'MB509-BEARING60002Z', 1, 'C1', 'D.13', 10.00, 100, 30, 1), -- TRUE becomes 1
('SP008', 'P008', 'MB862-HOSEPVCTRANSPARENT64X1MM', 3, 'C2', 'D.14', 15.00, 70, 20, 1), -- TRUE becomes 1
('SP009', 'P009', 'MAB39-NEEDLEFORPWINDING69MMLI', 3, 'C3', 'CC005', 7.50, 30, 10, 0), -- FALSE becomes 0
('SP010', 'P010', 'MC40B-SUPPLYPACKAGEPLATE', 1, 'C4', 'D.15', 22.00, 45, 15, 1), -- TRUE becomes 1
('SP011', 'P011', 'M5BF7-NAN', 2, 'D1', 'CC006', 30.00, 50, 20, 0); -- FALSE becomes 0

-- Insert data into machine_pos table
INSERT INTO machine_pos (mc_pos, mc_id)
VALUES
('POS1', 1),
('POS2', 1),
('POS3', 1),
('POS5', 1),
('FO1', 2),
('FO2', 2),
('FO3', 2),
('SM1', 3),
('SM2', 3),
('SM3', 3),
('GR1', 4),
('GR2', 4),
('GR3', 4),
('GR4', 4),
('GR5', 4),
('UGO1', 5),
('UGO2', 5),
('UGO3', 5),
('UGO4', 5);

-- Insert data into import_export table
INSERT INTO import_export (part_id, quantity, mc_pos_id, empl_id, date, reason, im_ex_flag)
VALUES
    ('SP001', 100, 'POS1', 'EMP001', '2025-05-10', 'Nhập hàng để thay thế', 1), -- TRUE becomes 1
    ('SP002', 50, 'FO1', 'EMP002', '2025-05-11', 'Xuất hàng cho bảo trì', 0), -- FALSE becomes 0
    ('SP003', 200, 'SM1', 'EMP003', '2025-05-12', 'Nhập hàng để bổ sung kho', 1), -- TRUE becomes 1
    ('SP004', 10, 'GR1', 'EMP004', '2025-05-13', 'Xuất hàng cho sửa chữa', 0), -- FALSE becomes 0
    ('SP005', 150, 'UGO1', 'EMP005', '2025-05-14', 'Nhập hàng từ nhà cung cấp', 1); -- TRUE becomes 1

-- Insert data into employees table with generated values for missing columns
-- Data for amann_id '23' is excluded as requested
INSERT INTO employees (amann_id, name, title, level, active, birthday, start_date, check_in_time, check_out_time, address, phone_number, email, gender, years_worked, age) VALUES
('EMP001', 'Nguyễn Văn A', 'Senior Engineer', 'Senior', 1, '1991-01-10', '2019-03-15', '08:00:00', '17:00:00', 'Tam Kỳ', '0901234501', 'nguyenvana@amann.com', 'Nam', NULL, NULL),
('EMP002', 'Trần Thị B', 'Middle Technician', 'Middle', 1, '1992-02-12', '2020-04-18', '08:00:00', '17:00:00', 'Đà Nẵng', '0901234502', 'tranthib@amann.com', 'Nữ', NULL, NULL),
('EMP003', 'Hoàng Trung', 'Lead Supervisor', 'Lead', 1, '1993-03-20', '2021-05-10', '08:30:00', '17:30:00', 'Tam Kỳ', '0901234503', 'hoangtrung@amann.com', 'Nam', NULL, NULL),
('EMP004', 'Lê Hoàng', 'Senior Specialist', 'Senior', 1, '1995-06-25', '2018-06-01', '08:00:00', '16:30:00', 'TP.HCM', '0901234504', 'lehoang@amann.com', 'Nam', NULL, NULL),
('EMP005', 'Ngọc Lan', 'Middle Operator', 'Middle', 1, '1990-07-15', '2020-01-15', '09:00:00', '18:00:00', 'Tam Kỳ', '0901234505', 'ngoclan@amann.com', 'Nữ', NULL, NULL),
('EMP006', 'Thanh Nguyên', 'Middle Engineer', 'Middle', 1, '1994-10-05', '2021-07-20', '08:00:00', '17:00:00', 'Số 16, Quận 6, TP.HCM', '0901234506', 'thanhnguyen@amann.com', 'Nam', NULL, NULL),
('EMP007', 'Khánh Hòa', 'Junior Technician', 'Junior', 1, '1996-11-23', '2022-08-30', '08:30:00', '17:30:00', 'Tam Kỳ', '0901234507', 'khanhhoa@amann.com', 'Nữ', NULL, NULL),
('EMP008', 'Đức Phát', 'Middle Specialist', 'Middle', 1, '1990-12-01', '2019-09-10', '08:00:00', '16:30:00', 'Đà Nẵng', '0901234508', 'ducphat@amann.com', 'Nam', NULL, NULL),
('EMP009', 'Tùng Tiến', 'Junior Operator', 'Junior', 1, '1995-09-18', '2021-04-01', '09:00:00', '18:00:00', 'Đà Nẵng', '0901234509', 'tungtien@amann.com', 'Nam', NULL, NULL),
('EMP010', 'Thùy Linh', 'Middle Engineer', 'Middle', 1, '1990-05-10', '2020-02-20', '08:30:00', '17:30:00', 'Huế', '0901234510', 'thuylinh@amann.com', 'Nữ', NULL, NULL),
('EMP011', 'Bảo Hiền', 'Junior Technician', 'Junior', 1, '1992-04-22', '2021-09-05', '08:00:00', '16:30:00', 'Huế', '0901234511', 'baohien@amann.com', 'Nữ', NULL, NULL);

-- Update age and years_worked for all inserted employees
-- Using SQLite functions strftime('%Y', 'now') for current year
UPDATE employees SET
    age = CAST(strftime('%Y', 'now') AS INTEGER) - CAST(strftime('%Y', birthday) AS INTEGER),
    years_worked = CAST(strftime('%Y', 'now') AS INTEGER) - CAST(strftime('%Y', start_date) AS INTEGER);
"""

In [7]:
conn = sqlite3.connect('warehouse.db')
cursor = conn.cursor()


# Chạy toàn bộ script một lần
cursor.executescript(insert_database)

conn.commit()
conn.close()